In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [4]:
def tokenize(setence):
    return nltk.word_tokenize(setence)

In [5]:
tokenize('how are you ????')

['how', 'are', 'you', '?', '?', '?', '?']

In [6]:
def stem(word):
    return stemmer.stem(word.lower())

In [7]:
stem('organic')

'organ'

In [16]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [17]:
bag_of_words(['lol','testing','programming','vr'],['lol','testing','programming','vr'])

array([1., 0., 0., 1.], dtype=float32)

In [18]:
data = pd.read_csv('./data.csv')

In [20]:
X = data['text']
y = data['target']

In [21]:
all_words = []
all_data = []
tags = []

In [23]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

7613it [00:02, 3097.09it/s]


In [27]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [28]:
all_words = sorted(set(all_words))

In [29]:
tags = sorted(set(tags))

In [30]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [31]:
X = []
y = []

In [32]:
for sent,tag in tqdm(all_data):
    bag = bag_of_words(sent,all_words)
    X.append(bag)
    tag = tags.index(tag)
    y.append(tag)

In [33]:
from sklearn.model_selection import *

In [34]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [ ]:
X_train = torch.from_numpy(np.array(X_train))

In [35]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds.view(-1,1),y.view(-1,1))
    return loss.item()

In [36]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(torch.round(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [37]:
class Model(Module):
    def __init__(self,hidden):
        super().__init__()
        self.activation = ReLU()
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden)
        self.linear3 = Linear(hidden,hidden)
        self.linear4 = Linear(hidden,hidden)
        self.linear5 = Linear(hidden,hidden)
        self.output = Linear(hidden,1)
        self.output_ac = Sigmoid()
    
    def forward(self,X):
        preds = self.activation(self.linear1(X))
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.output_ac(self.output(preds))
        return preds

In [39]:
device = 'cuda'

In [40]:
model = Model(512).to(device)
model.to(device)

Model(
  (activation): ReLU()
  (linear1): Linear(in_features=13928, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=512, bias=True)
  (linear4): Linear(in_features=512, out_features=512, bias=True)
  (linear5): Linear(in_features=512, out_features=512, bias=True)
  (output): Linear(in_features=512, out_features=1, bias=True)
  (output_ac): Sigmoid()
)

In [41]:
criterion = BCELoss()

In [42]:
optimizer = Adam(model.parameters(),lr=0.001)

In [44]:
epochs = 1000

In [45]:
batch_size = 32

In [46]:
PROJECT_NAME = 'NLP-Disaster-Tweets-Clf-4'

In [48]:
import wandb

In [49]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        preds = preds.to(device)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    wandb.watch(model)
wandb.watch(model)
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  0%|          | 0/1000 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'to'